In [13]:
from statsforecast import StatsForecast
from statsforecast.models import (
    CrostonClassic,
    CrostonSBA, 
    TSB,    
    ADIDA,     
    IMAPA,
)
from datasetsforecast.m5 import M5
df_total, *_ = M5.load('./data')

n_series = 8 
uids = df_total['unique_id'].unique()[:n_series]
df = df_total.query('unique_id in @uids')

In [50]:
dates = df['ds'].unique()[-28:] # last 28 days
train = df.query('ds not in @dates')
test = df.query('ds in @dates')

In [43]:
# Create a list of models and instantiation parameters 
models = [
    CrostonClassic(),
    CrostonSBA(),
    TSB(alpha_d=0.2, alpha_p=0.2),
    ADIDA(),
    IMAPA(),
]
sf = StatsForecast(
    df=train, 
    models=models, 
    freq='D',
    n_jobs=-1
)
horizon = 28 
forecasts = sf.forecast(h=horizon).reset_index()
forecasts.head()

,unique_id,ds,CrostonClassic,CrostonSBA,TSB,ADIDA,IMAPA
0,FOODS_1_001_CA_1,2016-05-23,0.898247,0.853334,0.434313,0.791852,0.705835
1,FOODS_1_001_CA_1,2016-05-24,0.898247,0.853334,0.434313,0.791852,0.705835
2,FOODS_1_001_CA_1,2016-05-25,0.898247,0.853334,0.434313,0.791852,0.705835
3,FOODS_1_001_CA_1,2016-05-26,0.898247,0.853334,0.434313,0.791852,0.705835
4,FOODS_1_001_CA_1,2016-05-27,0.898247,0.853334,0.434313,0.791852,0.705835


In [60]:
from sktime.performance_metrics.forecasting import MeanSquaredScaledError


rmsse = MeanSquaredScaledError(square_root=True)

def show_results(test, forecasts):
    test_df = test.merge(forecasts, how='left', on=['unique_id', 'ds'])
    cols = test_df.drop(columns=["unique_id", "ds", "y"])
    for col in cols:
        print(f'{col}: %0.3f' % rmsse(test_df[col], test_df.y, y_train=train.y))

show_results(test, forecasts)

CrostonClassic: 0.856
CrostonSBA: 0.860
TSB: 0.889
ADIDA: 0.860
IMAPA: 0.869


/opt/anaconda3/envs/time_series/lib/python3.10/site-packages/sktime/performance_metrics/forecasting/_classes.py:422: UserWarning: y_pred and y_true do not have the same column index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
/opt/anaconda3/envs/time_series/lib/python3.10/site-packages/sktime/performance_metrics/forecasting/_classes.py:422: UserWarning: y_pred and y_true do not have the same column index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
/opt/anaconda3/envs/time_series/lib/python3.10/site-packages/sktime/performance_metrics/forecasting/_classes.py:422: UserWarning: y_pred and y_true do not have the same column index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
/opt/anaconda3/envs/time_series/lib/python3.10/site-packages/sktime/performance_metrics/forecasting/_classes.py:422: UserWar